In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import json
import os

In [ ]:
def json_reader(begin):
    if begin not in np.arange(0, 11000, 1000):
        raise Exception(
            "Invalid start pid! Start pids must be {0, 1000, 2000, ..., 11000}"
        )

    end = begin + 999
    path = "../data/playlists/playlist.slice." + str(begin) + "-" + str(end) + ".json"

    jsonData = json.load(open(path, "r"))
    actualSlice = pd.DataFrame.from_dict(jsonData["playlists"], orient="columns")
    return actualSlice

In [ ]:
jsonList = []
for begin in np.arange(0, 11000, 1000):
    actual = json_reader(begin)
    jsonList.append(actual)

trainData = pd.concat(jsonList, ignore_index=True)
jsonList.clear()

print(trainData.shape)

In [ ]:
# Turn playlist level dataframe into song level dataframe
# is a df of all track ids, cooresponding artist names, track names and playlist ids

songPlaylistArray = []
for index, row in trainData.iterrows():
    for track in row["tracks"]:
        songPlaylistArray.append(
            [track["track_uri"], track["artist_name"], track["track_name"], row["pid"]]
        )
songPlaylist = pd.DataFrame(
    songPlaylistArray, columns=["trackid", "artist_name", "track_name", "pid"]
)
songPlaylistArray.clear()

print(songPlaylist.shape)
songPlaylist.head(10)

In [ ]:
def json_content_reader(begin):
    if begin not in np.arange(0, 731000, 1000):
        raise Exception(
            "Invalid start contentid! Start pids must be {0, 1000, 2000, ..., 731000}"
        )

    end = begin + 999
    path = (
        "../data/music_contents/content.slice." + str(begin) + "-" + str(end) + ".json"
    )
    jsonData = json.load(open(path, "r"))
    
    actualSlice = pd.DataFrame.from_dict(jsonData, orient="columns")
    return actualSlice

In [ ]:
jsonList = []
for begin in np.arange(0, 731000, 1000):
    actual = json_content_reader(begin)
    jsonList.append(actual)

contentData = pd.concat(jsonList, ignore_index=True)
jsonList.clear()

print(contentData.shape)
contentData.head(10)

In [ ]:
contentData.drop(
    columns=[
        "type",
        "track_href",
        "mode",
        "time_signature",
        "id",
        "analysis_url",
        "key",
        "duration_ms",
    ],
    inplace=True,
)

contentData.head(10)

In [ ]:
contentData = pd.merge(
    songPlaylist, contentData, left_on="trackid", right_on="uri", how="left"
)

In [ ]:
contentData.head(10)

In [ ]:
contentData.count()